In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
import torch.nn.init as init
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [2]:
N_train = 5000
N_bound = 200

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

x_inside: Column 0: range from -0.9999233922198485 to 0.9998822478677367
t_inside: Column 0: range from 0.0001244217542079664 to 0.9999681727021804
x_bound: Column 0: range from -0.9976551547696636 to 0.996510993325985
t_bound: Column 0: range from 0.0015792012865775578 to 0.9975818878215208


In [3]:
from scipy.io import loadmat

data = loadmat("usol_D_0.001_k_5.mat")

t_exact = data["t"].reshape(-1,1)

x_exact = data["x"].reshape(-1,1)

Exact_u = data["u"]

print('t_exact:',t_exact.shape)
print('x_exact:',x_exact.shape)
print('Exact_u:',Exact_u.shape)

t_exact: (101, 1)
x_exact: (201, 1)
Exact_u: (101, 201)


In [4]:
X_exact, T_exact = np.meshgrid(x_exact,t_exact)
print('X_exact:',X_exact.shape)
print('T_exact:',T_exact.shape)

X_exact_flatten = X_exact.flatten()[:,None]
T_exact_flatten = T_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None] 

X_exact: (101, 201)
T_exact: (101, 201)


In [5]:
data_star.shape

(20301, 2)

In [6]:
random_seed = 1234
np.random.seed(random_seed)
number_observe_data = 50

index_x = (lhs(1,number_observe_data))*len(x_exact)
index_x = np.floor(index_x).reshape(number_observe_data,).astype(int)
observe_x = x_exact[index_x]

index_t = (lhs(1,number_observe_data))*len(t_exact)
index_t = np.floor(index_t).reshape(number_observe_data,).astype(int)
observe_t = t_exact[index_t]

observe_data = np.hstack((observe_x,observe_t))
observe_clear_u = Exact_u[index_t,index_x].reshape(-1,1)

observe_u = observe_clear_u

observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_clear_u = numpy_to_tensor(observe_clear_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

observe_data: Column 0: range from -1.0 to 0.99
observe_data: Column 1: range from 0.01 to 0.99
observe_u: Column 0: range from -1.0 to 0.9528342844289072
observe_u: Column 0: range from -1.0 to 0.9528342844289072
J: 50


In [7]:
np.random.seed(5678)
n_test_data = 10000

index_x = (lhs(1,n_test_data))*len(x_exact)
index_x = np.floor(index_x).reshape(n_test_data,).astype(int)
observe_x = x_exact[index_x]

index_t = (lhs(1,n_test_data))*len(t_exact)
index_t = np.floor(index_t).reshape(n_test_data,).astype(int)
observe_t = t_exact[index_t]

test_data = np.hstack((observe_x,observe_t))
test_u = Exact_u[index_t,index_x].reshape(-1,1)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

test_data: Column 0: range from -1.0 to 1.0
test_data: Column 1: range from 0.0 to 1.0
test_u: Column 0: range from -1.0 to 0.9918199944816758


In [8]:
C1 = torch.tensor(0.5, requires_grad=True)

In [9]:
print('J:',len(observe_u))

J: 50


In [10]:
def output_transform(data_input, u_input):
    
    x_in = data_input[:,0:1]
    t_in = data_input[:,1:2]
    
    return x_in**2 * torch.cos(torch.tensor(np.pi) * x_in) + t_in * (1 - x_in**2) * u_input

In [11]:
def get_loss_f(x_grad, y_grad, PINNs, C, return_sequence='not'):

    E_inside = PINNs(torch.cat((x_grad,y_grad),axis=1))
    E_inside = output_transform(torch.cat((x_grad,y_grad),axis=1),E_inside)

    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])
    E_xxx = compute_higher_order_derivatives(E_xx, [x_grad])
    E_xxt = compute_higher_order_derivatives(E_xx, [y_grad])
    E_t = compute_higher_order_derivatives(E_inside, [y_grad])
    E_tx = compute_higher_order_derivatives(E_t, [x_grad])
    E_tt = compute_higher_order_derivatives(E_t, [y_grad])

    loss_term = E_t - C * E_xx - 5 * (E_inside - E_inside**3)

    if return_sequence == 'yes':
        return torch.square(loss_term)
    else:
        return torch.mean(torch.square(loss_term))

In [12]:
def get_loss_bound(bound_x, bound_t, PINNs, C, return_sequence='not'):

    E_bound_x_zero = PINNs(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1)) 
    Exact_x_zero = bound_x**2*torch.cos(torch.tensor(np.pi)*bound_x)
    loss_bound_for_a = torch.mean(torch.square(E_bound_x_zero-Exact_x_zero))
    
    E_bound_fu_1_t = PINNs(torch.cat((-torch.ones_like(bound_t),bound_t),axis=1)) 
    Exact_fu_1_t = -torch.ones_like(E_bound_fu_1_t)
    loss_bound_for_b = torch.mean(torch.square(E_bound_fu_1_t-Exact_fu_1_t))
    
    E_bound_1_t = PINNs(torch.cat((torch.ones_like(bound_t),bound_t),axis=1))
    Exact_1_t = -torch.ones_like(E_bound_1_t)
    loss_bound_for_c = torch.mean(torch.square(E_bound_1_t-Exact_1_t))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c
    
    return loss_bound_value

In [13]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [14]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[64]*6, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [15]:
x_inside_all = torch.cat((x_inside,observe_data[:,0:1]),axis=0)
t_inside_all = torch.cat((t_inside,observe_data[:,1:2]),axis=0)

In [16]:
net_settings_for_discriminator1 = NetSetting(input_dims=3, hidden_neurons_list=[64]*3, 
                                     output_dims=1, hidden_activation='sigmoid(tanh)', 
                                     output_activation='sigmoid(tanh)', initializer_method='xavier')

discriminator1 = get_mlp_pinn(net_settings_for_discriminator1)
discriminator1.cuda() 
initialize_weights(discriminator1, net_settings_for_discriminator1.initializer_method)

optimizer2 = optim.Adam(discriminator1.parameters(), lr=1e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)

In [17]:
def train_optimizer1(PINNs,C,discriminator):
    
    for param in PINNs.parameters():
        param.requires_grad = True
    
    C.requires_grad = True
    
    for param in discriminator.parameters():
        param.requires_grad = False

In [18]:
def train_optimizer2(PINNs,C,discriminator):
    
    for param in PINNs.parameters():
        param.requires_grad = False
    
    C.requires_grad = False
    
    for param in discriminator.parameters():
        param.requires_grad = True

In [19]:
def train_for_gan(PINNs,C,discriminator,observe_data_input,observe_u_input):
    
    
    ############ loss D ###########
    train_optimizer2(PINNs,C,discriminator)
    
    pre_H = PINNs(observe_data_input)
    pre_H = output_transform(observe_data_input,pre_H)
    
    d_fake = discriminator(torch.cat((observe_data_input,pre_H),1))
    d_real = discriminator(torch.cat((observe_data_input,observe_u_input),1))

    loss_D = torch.mean(1-d_real)+torch.mean(d_fake)

    optimizer2.zero_grad()
    loss_D .backward()
    optimizer2.step()  

    ############ loss G ###########
    train_optimizer1(PINNs,C,discriminator)
    
    pre_H = PINNs(observe_data_input)
    pre_H = output_transform(observe_data_input,pre_H)
    
    d_fake = discriminator(torch.cat((observe_data_input,pre_H),1))

    loss_G = torch.mean(1-d_fake)+torch.mean(torch.square(pre_H - observe_u_input))

    optimizer1.zero_grad()
    loss_G.backward()
    optimizer1.step()  
    
    return PINNs,C,discriminator

In [20]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
test_loss_1 = []
C1_list = []
############## Record list ###############


nIter1 = 10000
it = 0

while it<nIter1:
    
    if it<4000:
        ######### GAN  #########
        PINNs1,C1,discriminator1 = train_for_gan(PINNs1,C1,discriminator1,observe_data,observe_u)    
        
    #########loss f#########
    loss_f = get_loss_f(x_inside_all,t_inside_all,PINNs1,C1,return_sequence='not')

    #########loss T observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation)
    loss_observation = torch.mean(torch.square(E_observation-observe_u))            
    
    #########loss PI#########
    loss = loss_f+10*loss_observation
    
    #########test_loss NRMSE#########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data,pre_u)
    test_loss = relative_l2_torch(pre_u,test_u)
    #########test_loss NRMSE#########
    
    #########Record#########
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation.item()) 
    test_loss_1.append(test_loss)
    #########Record#########
    
    if it % 1000 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        print(C1)
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    
    it = it + 1   
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)

It: 0 train_loss: 8.277297973632812 test_loss: 0.6611247062683105
tensor(0.5000, requires_grad=True)
It: 1000 train_loss: 0.018136000260710716 test_loss: 0.07932429015636444
tensor(0.0032, requires_grad=True)
It: 2000 train_loss: 0.014250827953219414 test_loss: 0.05685986205935478
tensor(0.0017, requires_grad=True)
It: 3000 train_loss: 0.00538280513137579 test_loss: 0.03836940973997116
tensor(0.0012, requires_grad=True)
It: 4000 train_loss: 0.0014072509948164225 test_loss: 0.026851344853639603
tensor(0.0011, requires_grad=True)
It: 5000 train_loss: 0.0005835536867380142 test_loss: 0.015040514059364796
tensor(0.0011, requires_grad=True)
It: 6000 train_loss: 0.0017009468283504248 test_loss: 0.016024192795157433
tensor(0.0011, requires_grad=True)
It: 7000 train_loss: 0.0005161423468962312 test_loss: 0.010685734450817108
tensor(0.0011, requires_grad=True)
It: 8000 train_loss: 0.00032323499908670783 test_loss: 0.008287093602120876
tensor(0.0011, requires_grad=True)
It: 9000 train_loss: 0.00

In [21]:
import os
folder_path = '../experimental_data/GA_PINN/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(os.path.join(folder_path, 'loss_T_1.npy'), loss_T_1)
np.save(os.path.join(folder_path, 'test_loss_1.npy'), test_loss_1)
np.save(os.path.join(folder_path, 'C1_list.npy'), C1_list)